# Acoustoelectric signal detection feasibility

### Signal calculation
Starting from 100 µV EEG signal (0-100 Hz bandwidth):
- Acoustoelectric coefficient: $K_{AE} = 4.3 \times 10^{-10} \text{ Pa}^{-1}$
- Ultrasound pressure: 1 MPa
- Focus area scaling: 5×5 mm spot vs 20×20 mm EEG pickup area = 25/400

$$V_{AE} = 100 \text{ µV} \times 10^{-9} \text{ Pa}^{-1} \times 10^{6} \text{ Pa} \times \frac{25}{400} = 2.69 \text{ nV}$$

For 5×5 mm focused ultrasound: **2.69 nV**

### Thermal noise calculation
Johnson noise: $V_{noise} = \sqrt{4k_BTR\Delta f}$

With common reference electrode path resistance R = 1 kΩ:

In [113]:
import numpy as np

# Constants
k_B = 1.38e-23  # J/K
T = 310  # K
V_EEG_sig = 100000 # nV
bandwidth = 100  # Hz
pickup_area = 400
focal_area = 25
p = 1e6 # Pa
k = 4.3e-10 # 1/Pa from https://pmc.ncbi.nlm.nih.gov/articles/PMC3476726/

# Calculate noise for different resistances
R_values = [100, 500, 1000, 2000]  # Ohms
for R in R_values:
    V_noise = np.sqrt(4 * k_B * T * R * bandwidth) * 1e9  # nV
    print(f"R = {R:4d} Ω: V_noise = {V_noise:.1f} nV")

# Signal vs noise
V_signal = V_EEG_sig*k*p*(focal_area/pickup_area)  # nV
print(f"\nSignal: {V_signal:.3f} nV")
print(f"SNR at 1kΩ: {20*np.log10(V_signal/4.1):.1f} dB")

R =  100 Ω: V_noise = 13.1 nV
R =  500 Ω: V_noise = 29.3 nV
R = 1000 Ω: V_noise = 41.4 nV
R = 2000 Ω: V_noise = 58.5 nV

Signal: 2.688 nV
SNR at 1kΩ: -3.7 dB


### Noise averaging with multiple electrode pairs

If each active electrode has its own reference electrode, the reference noise becomes uncorrelated between channels.

In [114]:
# SNR improvement with N independent electrode-reference pairs
N_pairs = np.array([1, 2, 4, 8, 16, 32])
R_ref = 2000  # Ohms per electrode

# Noise reduces as 1/sqrt(N) for uncorrelated sources
V_noise_single = np.sqrt(4 * k_B * T * R_ref * bandwidth) * 1e9
V_noise_averaged = V_noise_single / np.sqrt(N_pairs)

print("Pairs | Noise (nV) | SNR (dB)")
print("------|------------|----------")
for n, v_n in zip(N_pairs, V_noise_averaged):
    snr = 20 * np.log10(V_signal / v_n)
    print(f"{n:5d} | {v_n:10.2f} | {snr:8.1f}")

Pairs | Noise (nV) | SNR (dB)
------|------------|----------
    1 |      58.50 |    -26.8
    2 |      41.37 |    -23.7
    4 |      29.25 |    -20.7
    8 |      20.68 |    -17.7
   16 |      14.63 |    -14.7
   32 |      10.34 |    -11.7


### Common average reference (CAR)

For N channels sharing a common reference with correlation factor ρ:

In [115]:
# CAR noise reduction
def car_noise_reduction(N_channels, rho=0.7):
    """Residual noise fraction after CAR"""
    return np.sqrt((1 - rho) + rho/N_channels)

N_channels = [4, 8, 16, 32, 64, 128]
rho_values = [0.5, 0.7, 0.9]

print("Channels |   ρ=0.5  |   ρ=0.7  |   ρ=0.9")
print("---------+----------+----------+---------")
for N in N_channels:
    reductions = [car_noise_reduction(N, rho) for rho in rho_values]
    print(f"{N:8d} | {reductions[0]:.2f}     | {reductions[1]:.2f}     | {reductions[2]:.2f}")

Channels |   ρ=0.5  |   ρ=0.7  |   ρ=0.9
---------+----------+----------+---------
       4 | 0.79     | 0.69     | 0.57
       8 | 0.75     | 0.62     | 0.46
      16 | 0.73     | 0.59     | 0.40
      32 | 0.72     | 0.57     | 0.36
      64 | 0.71     | 0.56     | 0.34
     128 | 0.71     | 0.55     | 0.33


### Temporal averaging

For repeated measurements with white noise:

In [116]:
# Temporal averaging SNR improvement
n_averages = [1, 10, 100, 1000, 10000]
base_noise = 4.1  # nV at 1kΩ

print("Averages | Noise (nV) | SNR gain (dB)")
print("---------+------------+--------------")
for n in n_averages:
    noise_reduced = base_noise / np.sqrt(n)
    gain_db = 10 * np.log10(n)
    print(f"{n:8d} | {noise_reduced:10.3f} | {gain_db:12.1f}")

Averages | Noise (nV) | SNR gain (dB)
---------+------------+--------------
       1 |      4.100 |          0.0
      10 |      1.297 |         10.0
     100 |      0.410 |         20.0
    1000 |      0.130 |         30.0
   10000 |      0.041 |         40.0


### Combined approach

Stacking all techniques:

In [117]:
# Calculate total SNR with combined techniques
R_base = 1000  # Ohms

# Base noise
V_noise_base = np.sqrt(4 * k_B * T * R_base * bandwidth) * 1e9

# Apply improvements
n_ref_pairs = 4
car_channels = 32
car_correlation = 0.7
n_time_avg = 1000

# Calculate cumulative noise reduction
noise_after_refs = V_noise_base / np.sqrt(n_ref_pairs)
noise_after_car = noise_after_refs * car_noise_reduction(car_channels, car_correlation)
noise_final = noise_after_car / np.sqrt(n_time_avg)

print(f"Initial noise: {V_noise_base:.1f} nV")
print(f"After {n_ref_pairs} ref pairs: {noise_after_refs:.2f} nV")
print(f"After {car_channels}-ch CAR: {noise_after_car:.3f} nV")
print(f"After {n_time_avg}x averaging: {noise_final:.4f} nV")
print(f"\nSignal: {V_signal:.4f} nV")
print(f"Final SNR: {20*np.log10(V_signal/noise_final):.1f} dB")

Initial noise: 41.4 nV
After 4 ref pairs: 20.68 nV
After 32-ch CAR: 11.734 nV
After 1000x averaging: 0.3711 nV

Signal: 2.6875 nV
Final SNR: 17.2 dB


### Requirements for detection

Calculate required noise for 20 dB SNR.

Required averaging to achieve this from base noise:

In [118]:
# Calculate required averaging
target_noise = V_signal / 10  # 20 dB SNR
base_noise = 4.1  # nV
print(target_noise)

# With various improvements
scenarios = [
    ("Single channel", base_noise),
    ("4 ref pairs", base_noise / 2),
    ("4 refs + 32ch CAR", base_noise / 2 * 0.57),
    ("4 refs + 128ch CAR", base_noise / 2 * 0.52)
]

print("Scenario | Starting noise | Required averages")
print("---------+----------------+------------------")
for name, noise_start in scenarios:
    n_required = (noise_start / target_noise) ** 2
    print(f"{name:20s} | {noise_start:6.2f} nV | {n_required:12.0f}")

0.26875
Scenario | Starting noise | Required averages
---------+----------------+------------------
Single channel       |   4.10 nV |          233
4 ref pairs          |   2.05 nV |           58
4 refs + 32ch CAR    |   1.17 nV |           19
4 refs + 128ch CAR   |   1.07 nV |           16
